In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [2]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [3]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [4]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [5]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [6]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [7]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
  7/400 [..............................] - ETA: 3s - loss: 1.6386 - accuracy: 0.2991 

2024-03-29 21:55:54.865933: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


400/400 [==============================] - 3s 6ms/step - loss: 1.3318 - accuracy: 0.4914 - val_loss: 0.8229 - val_accuracy: 0.7266
Epoch 2/10
400/400 [==============================] - 3s 7ms/step - loss: 0.3224 - accuracy: 0.9010 - val_loss: 0.5430 - val_accuracy: 0.8159
Epoch 3/10
400/400 [==============================] - 3s 7ms/step - loss: 0.0596 - accuracy: 0.9834 - val_loss: 0.5951 - val_accuracy: 0.8234
Epoch 4/10
400/400 [==============================] - 3s 7ms/step - loss: 0.0251 - accuracy: 0.9947 - val_loss: 0.5919 - val_accuracy: 0.8284
Epoch 5/10
400/400 [==============================] - 3s 7ms/step - loss: 0.0159 - accuracy: 0.9966 - val_loss: 0.6266 - val_accuracy: 0.8284
Epoch 6/10
400/400 [==============================] - 3s 7ms/step - loss: 0.0147 - accuracy: 0.9967 - val_loss: 0.6427 - val_accuracy: 0.8234
Epoch 7/10
400/400 [==============================] - 3s 7ms/step - loss: 0.0109 - accuracy: 0.9977 - val_loss: 0.6361 - val_accuracy: 0.8181
Epoch 8/10
400/40

In [9]:
input_text = "It was fun"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 14ms/step
['joy']
